In [3]:
#Setup inference and evaluation parameters
import re
import sys
import os
import random
GPU_ID="0"
os.environ["CUDA_VISIBLE_DEVICES"]=GPU_ID

Customer=1
CustomPrompt=1
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
import Libs.DataSynthesisLib as DataSynthesisLib
Config=DataSynthesisLib.GetConfig('Config/Config.py')

ModelFT='/tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05'
ModelBase='unsloth/Meta-Llama-3.1-8B-Instruct'


AssistantPromptElementsRemoval=2
EvalLLM='microsoft/phi-4'


In [4]:
#Load appropriate evaluation dataset
import json
import random


if Customer==1:
  with open('Data/CustomerComments_Evaluation_Clean.json', 'r') as file:
    data = json.load(file)
elif Customer==0:
  with open('Data/ClouderaComments_Evaluation_Clean.json', 'r') as file:
    data = json.load(file)





In [5]:
#Setup questions for LLM-as-a-judge, collect prompts for inference, and customer or Cloudera comments to be used later for evaluation
import vllm
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(ModelFT)

System='You are a helpful assistant.'


FixedQuestions={}
if Customer==1:
  FixedQuestions[1]="Does this comment discuss any technical information? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[2]="Does this comment relate to a customer complaint? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[3]="Customer complaint temperature or a frustration level (if there is a complain give a Score 1-4, lowest=1, highest=4. If there is no complain give a score of 0)"
  FixedQuestions[4]="Score the severity of the issue based on comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[5]="Score the urgency of the issue based on the comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[6]="Is this a request from a customer for an update? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[7]="Is there a strictly explicit and NOT an implied request from a customer for a call, meeting or a screenshare (zoom/webex/teams etc.)? Do not answer yes unless wording explicitly asks for a call. (BOOL)"
  FixedQuestions[8]="Did the customer request an escalation? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[9]="Did the customer request a priority change?  To what level? (If there is a priority change give score 1 to indicate highest priority (indicated by S1) and 4  to indicate the lowest priority (Indicated by 4). If there is no priority change give a score of 0)."
  FixedQuestions[10]="Did the customer request a transfer to another COE? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[11]="Did the customer request to speak to a manager or supervisor? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[12]="Did the customer request a SME or expert? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[13]="Does this comment discuss a bug in Cloudera software? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[14]="Does the comment include a non-Cloudera Apache JIRA link? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[15]="Does the comment have a link to Cloudera Documentation or Community article? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[16]="Does the comment have any other type of hyperlink? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[17]="Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT]"
elif Customer==0:
  FixedQuestions[1]="Does this comment discuss any technical information? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[2]="Score the severity of the issue based on comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[3]="Score the urgency of the issue based on the comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[4]="Does the comment have a proposed solution? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[5]="Does the comment have a proposed workaround?  (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[6]="Does the comment have a request for an action from the customer?  (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[7]="Does this comment discuss a bug in Cloudera software? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[8]="Does the comment include a non-Cloudera Apache JIRA link? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[9]="Does the comment have a link to Cloudera Documentation or Community article? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[10]="Does the comment have any other type of hyperlink? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[11]="Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT] "


Questions=[]
Comment=[]
BaselineSolutions=[]
for i in data:
  if Customer==1:
      
    if CustomPrompt==1:

      Questions.append(i['Prompt'])
      Comment.append(i['Comment'])

  elif Customer==0:

    CommentText="""You are given a Cloudera support team comment, 11 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the Cloudera comment:

"""
    PreppendClouderaQuestions="""

Here are the 11 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Score the severity of the issue based on comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
3. Score the urgency of the issue based on the comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
4. Does the comment have a proposed solution? (answer 0 for no, 1 for yes)
5. Does the comment have a proposed workaround?  (answer 0 for no, 1 for yes)
6. Does the comment have a request for an action from the customer?  (answer 0 for no, 1 for yes)
7. Does this comment discuss a bug in Cloudera software? (answer 0 for no, 1 for yes)
8. Does the comment include a non-Cloudera Apache JIRA link (e.g. a Apache JIRA link with issues.apach.org domain name)? (answer 0 for no, 1 for yes)
9. Does the comment have a link to Cloudera Documentation or Community article? (answer 0 for no, 1 for yes)
10. Does the comment have any other type of hyperlink? (answer 0 for no, 1 for yes)
11. Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT]    
Generate each answer in a new line using the the format "Number. answer" (e.g. 1. 0)
"""
    if CustomPrompt==1:
      Questions.append(i['Prompt'])
      Comment.append(i['Comment'])


print(len(Comment))
print(len(Questions))
lens=[len(i) for i in Questions ]

500
500


In [6]:
#Typical example on how the LLM is prompted
print(Questions[0])

You are given a customer comment, 17 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the customer comment:

Hi Cloudera Support Team,

I noticed that our Ranger policies are not synchronizing properly across our cluster nodes. Several users are reporting access denied errors for resources they should have permissions to access. The Ranger Admin UI shows the policies as properly configured, but they're not being enforced correctly on the services. We've already tried restarting the Ranger admin service and clearing the policy cache, but the issue persists.

This is impacting our team's ability to access critical data resources.

Best regards,
Mark

Here are the 17 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Does this comment relate to a customer complaint? (answer 0 for no, 1 for yes)
3. Customer complaint temperature or a frustration level (if there is a c

In [7]:
#Base model parameters setup and inference 
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
ConfigS=Config["ConfigSolution"].copy()
ConfigS["gpu-id"]=GPU_ID
ConfigS["tensor_parallel_size"]="2"
ConfigS["tensor_parallel_size"]="1"

ConfigS["system"]=System
ConfigS["use-grammar"]="False"
ConfigS["AssistantPrompt"]=""

ConfigS["max_tokens"]="3000"
ConfigS["temperature"]="0.1"
ConfigS["max_num_seqs"]="2"

ConfigS["max_num_seqs"]="16"
ConfigS["max_model_len"]="32000"
ConfigS["max_num_batched_tokens"]="32000"


ConfigS["AssistantPromptElementsRemoval"]="1"
ConfigS["llm-path"]=ModelBase
ConfigS["tokenizer-path"]=ModelBase

ConfigS["tokenizer-file-gguf"]=""
ConfigS["temperature"]="0"
if "mistral" in ModelBase:
  ConfigS["GenerateChatTemplate"]="GenerateChatWithoutSystemWithAssistant"
else:
  ConfigS["GenerateChatTemplate"]="GenerateChatWithSystem"

ConfigS["UseOutlines"]="False"
ConfigS["AssistantPromptElementsRemoval"]="1"
ConfigS["tokenizer-file-gguf"]=""
if 'gptq' in ModelFT:
  ConfigS["quantization"]="gptq"
ConfigS["gpu_memory_utilization"]="0.7"


(SolutionsBase, SolutionsBaseLogging)=DataSynthesisLib.GetSolutions(ConfigS, Questions, ConfigS["system"])


/home/cdsw/.local/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


unsloth/Meta-Llama-3.1-8B-Instruct
WARNING 03-21 19:18:40 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-21 19:18:40 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='unsloth/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='unsloth/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.36it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:00,  2.05it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.72it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.48it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.56it/s]



INFO 03-21 19:18:45 model_runner.py:1071] Loading model weights took 14.9888 GB
INFO 03-21 19:18:48 gpu_executor.py:122] # GPU blocks: 6230, # CPU blocks: 0
INFO 03-21 19:18:48 gpu_executor.py:126] Maximum concurrency for 32000 tokens per request: 3.12x


Processed prompts: 100%|██████████| 500/500 [02:20<00:00,  3.55it/s, est. speed input: 2688.18 toks/s, output: 466.66 toks/s]


In [8]:
#Finetuned model parameters setup and inference 

from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
ConfigS=Config["ConfigSolution"].copy()
ConfigS["gpu-id"]=GPU_ID
ConfigS["tensor_parallel_size"]="2"
ConfigS["tensor_parallel_size"]="1"

ConfigS["system"]=System
ConfigS["use-grammar"]="False"
ConfigS["AssistantPrompt"]=""

ConfigS["max_tokens"]="3000"
ConfigS["temperature"]="0.1"
ConfigS["max_num_seqs"]="2"

ConfigS["max_num_seqs"]="16"
ConfigS["max_model_len"]="32000"
ConfigS["max_num_batched_tokens"]="32000"


ConfigS["AssistantPromptElementsRemoval"]="1"
ConfigS["llm-path"]=ModelFT
ConfigS["tokenizer-path"]=ModelFT

ConfigS["tokenizer-file-gguf"]=""
ConfigS["temperature"]="0"
if "mistral" in ModelFT:

  ConfigS["GenerateChatTemplate"]="GenerateChatWithoutSystemWithAssistant"
else:
  ConfigS["GenerateChatTemplate"]="GenerateChatWithSystem"

ConfigS["UseOutlines"]="False"
ConfigS["AssistantPromptElementsRemoval"]="1"
ConfigS["tokenizer-file-gguf"]=""
if 'gptq' in ModelFT:
  ConfigS["quantization"]="gptq"
ConfigS["gpu_memory_utilization"]="0.7"


(Solutions, SolutionsLogging)=DataSynthesisLib.GetSolutions(ConfigS, Questions, ConfigS["system"])


/home/cdsw/.local/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


/tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05
WARNING 03-21 19:21:38 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-21 19:21:38 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='/tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05', speculative_config=None, tokenizer='/tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observabi

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  6.37it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.27it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.61it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.44it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.64it/s]



INFO 03-21 19:21:43 model_runner.py:1071] Loading model weights took 14.9888 GB
INFO 03-21 19:21:46 gpu_executor.py:122] # GPU blocks: 6230, # CPU blocks: 0
INFO 03-21 19:21:46 gpu_executor.py:126] Maximum concurrency for 32000 tokens per request: 3.12x


Processed prompts: 100%|██████████| 500/500 [02:16<00:00,  3.66it/s, est. speed input: 2771.73 toks/s, output: 450.90 toks/s]


In [25]:
#Typical example on how the LLM is prompted
print(Questions[43])

You are given a customer comment, 17 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the customer comment:

Hi Cloudera Support Team,

I'm experiencing a critical issue with our CDE Virtual Cluster where our nightly data processing jobs are failing with serialization errors. The jobs process around 500GB of data and involve complex transformations. The error occurs during the shuffle phase with 'java.io.NotSerializableException'. We've tried increasing the serializer buffer size but the issue persists. This is impacting our daily reporting requirements.

Best regards,
Karen

Here are the 17 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Does this comment relate to a customer complaint? (answer 0 for no, 1 for yes)
3. Customer complaint temperature or a frustration level (if there is a complain give 1 for lowest, 4 for highest and 2,3 for in between. If there 

In [10]:
#Example finetuned model output
print(Solutions[43])

<|start_header_id|>assistant<|end_header_id|>

1. 1
2. 1
3. 3
4. 4
5. 4
6. 0
7. 0
8. 0
9. 0
10. 0
11. 0
12. 0
13. 1
14. 0
15. 0
16. 0
17. Customer reports critical issues with CDE Virtual Cluster where nightly data processing jobs (500GB) fail with serialization errors during shuffle phase, despite increasing serializer buffer size.


In [11]:
#Individual questions for evaluating each question using LLM-as-a-judge
FixedQuestions

{1: 'Does this comment discuss any technical information? (BOOL: 0 for no, 1 for yes)',
 2: 'Does this comment relate to a customer complaint? (BOOL: 0 for no, 1 for yes)',
 3: 'Customer complaint temperature or a frustration level (if there is a complain give a Score 1-4, lowest=1, highest=4. If there is no complain give a score of 0)',
 4: 'Score the severity of the issue based on comment content (SCORE 1-4, lowest=1, highest=4)',
 5: 'Score the urgency of the issue based on the comment content (SCORE 1-4, lowest=1, highest=4)',
 6: 'Is this a request from a customer for an update? (BOOL: 0 for no, 1 for yes)',
 7: 'Is there a strictly explicit and NOT an implied request from a customer for a call, meeting or a screenshare (zoom/webex/teams etc.)? Do not answer yes unless wording explicitly asks for a call. (BOOL)',
 8: 'Did the customer request an escalation? (BOOL: 0 for no, 1 for yes)',
 9: 'Did the customer request a priority change?  To what level? (If there is a priority change

In [12]:
#Parse LLM output and extract analytics and summary
def FormatOutput(Solutions,StartLine=3,ExpectedQuestions=17):
  CorrectFormat=0
  Output=[]
  Correct=[]

  for d in Solutions:
    result=d.split('\n')
    QuestionsAnswers={}
    count=0
    CorrectFormat=0
      
    for j in result[StartLine-1:]:
      FoundNumber=0
      count+=1
      context=""
      number=j.split('.')[0]
      if number.isdigit():
          number=int(number)
          if number==count:
              FoundNumber=1
      tmp=j.split(' ')
      if len(tmp)>=2:
         if count<ExpectedQuestions:
             if tmp[1].isdigit():
                 context=int(tmp[1])
                 if FoundNumber==1:
                    CorrectFormat+=1
                    QuestionsAnswers[number]=context
         else:
             context=' '.join(tmp[1:])
             if FoundNumber==1:
                CorrectFormat+=1
                QuestionsAnswers[number]=context
    Correct.append(CorrectFormat==ExpectedQuestions)
    Output.append(QuestionsAnswers)
  return (Output,Correct)

StartLine=3
if "Qwen2.5" in ModelFT:
    StartLine=2
if "mistral" in ModelFT:
    StartLine=1

if Customer==1:
  (FTOutput,FTCorrectFormat)=FormatOutput(Solutions,StartLine=StartLine)
elif Customer==0:
  (FTOutput,FTCorrectFormat)=FormatOutput(Solutions,StartLine=StartLine,ExpectedQuestions=11)



{1: 1, 2: 1, 3: 3, 4: 4, 5: 4, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 'Customer reports critical performance issues in CDP Enterprise after patch installation, with Hbase operations timing out or failing, showing increased GC pauses and high heap usage, impacting multiple critical applications.'}
500
500


485

In [ ]:
#Parsed formatted output for the finetuned LLM
print(FTOutput[12])
print(len(FTOutput))
print(len(FTCorrectFormat))
sum(FTCorrectFormat)

In [24]:
#Parsed formatted output for the finetuned LLM
Solutions[2].split('\n')[2:]

['1. 1',
 '2. 1',
 '3. 2',
 '4. 3',
 '5. 3',
 '6. 0',
 '7. 0',
 '8. 0',
 '9. 0',
 '10. 0',
 '11. 0',
 '12. 0',
 '13. 0',
 '14. 0',
 '15. 0',
 '16. 0',
 '17. Customer reports PySpark jobs failing with garbage collection errors in CDE after increasing data processing volume from 1TB to 3TB daily. Jobs are experiencing executor losses with current configuration of 12GB executor memory and 6 cores per executor. Requesting guidance on optimal GC and memory settings.']

In [14]:
StartLine=3
if "Qwen2.5" in ModelBase:
    StartLine=2
if "mistral" in ModelBase:
    StartLine=1

if Customer==1:
  (BaselineOutput,BaselineCorrectFormat)=FormatOutput(SolutionsBase,StartLine=StartLine)
elif Customer==0:
  (BaselineOutput,BaselineCorrectFormat)=FormatOutput(SolutionsBase,StartLine=StartLine,ExpectedQuestions=11)



{1: 1, 2: 1, 3: 2, 4: 3, 5: 3, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: "We're experiencing issues with our PySpark jobs in CDE where they're randomly failing with garbage collection errors after increasing our data processing volume from 1TB to 3TB daily."}
500
500


500

In [ ]:
#Parsed formatted output for the base LLM

print(BaselineOutput[2])
print(len(BaselineOutput))
print(len(BaselineCorrectFormat))
sum(BaselineCorrectFormat)

In [15]:
#Prompts and 3 examples for LLM-as-a-judge to score whether model A or B wins or if there is a tie.


comment="Hi Support team, we're experiencing critical performance issues with our Hive queries in our production environment. All queries are taking 3-4 times longer than usual to execute, severely impacting our business operations. Can you please escalate this to a high priority case and have someone look into this ASAP? We need immediate assistance as this is affecting our SLAs."
question="""Does this comment relate to a customer complaint? (BOOL)"""
AnswerA="""0"""
AnswerB="""1"""

Example1Prompt="You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + comment +".\nHere is the question:\n" + question+":\n" +"\nHere is the answer of System A:\n"+ AnswerA +"\nHere is the answer of System B:\n"+AnswerB

Example1Completion="""```json
{
    "choice": "B"
}
```
"""

comment="We're experiencing issues with our Cloudera Data Warehouse environment where our Impala queries are failing with OOM errors. The cluster metrics show high memory utilization across all nodes. This is blocking our critical reporting pipeline. Could you please help investigate this issue? We need this resolved within the next 24 hours."
question="""Customer complaint temperature or a frustration level (score 1-4, 0 if not a complaint)"""
AnswerA="""3"""
AnswerB="""3"""

Example2Prompt="You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + comment +".\nHere is the question:\n" + question+":\n" +"\nHere is the answer of System A:\n"+ AnswerA +"\nHere is the answer of System B:\n"+AnswerB

Example2Completion="""```json
{
    "choice": "tie"
}
```
"""
comment="Our Cloudera Manager is showing multiple critical alerts for HDFS data nodes. The replication factor has dropped below the minimum threshold, and we're seeing increased latency in our data processing jobs. Can you please help us resolve this urgently? This is impacting our production workloads."
question="""Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT]"""
AnswerA="""Critical HDFS issues with data nodes showing alerts, reduced replication factor causing increased latency in production data processing jobs."""
AnswerB="""This is about critical alets."""

Example3Prompt="You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + comment +".\nHere is the question:\n" + question+":\n" +"\nHere is the answer of System A:\n"+ AnswerA +"\nHere is the answer of System B:\n"+AnswerB
Example3Completion="""```json
{
    "choice": "A"
}
```
"""



In [16]:

chat2 = [
  {"role": "system", "content": System},

  {"role": "user", "content": Example1Prompt},
  {"role": "assistant", "content": Example1Completion},
  {"role": "user", "content": Example2Prompt},
  {"role": "assistant", "content": Example2Completion},
  {"role": "user", "content": Example3Prompt},
  {"role": "assistant", "content": Example3Completion}
]

Completion="""```json
{
    "choice": """

chat=chat2.copy()


In [17]:
#Setup LLM-as-a-judge final prompt
from transformers import AutoTokenizer
EvalLLM='microsoft/phi-4'
RemoveAssistant=2
if EvalLLM=='microsoft/phi-4':
    RemoveAssistant=1
elif EvalLLM=='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int8':
    RemoveAssistant=2
#tokenizer = AutoTokenizer.from_pretrained(EvalLLM)
tokenizer = AutoTokenizer.from_pretrained(EvalLLM)

if Customer==1:   
    ExpectedQuestions=17

elif Customer==0:
    ExpectedQuestions=11
    
FTCorrectFormat={}
BaselineCorrectFormat={}
AllPrompts=[]

for i in range(1,ExpectedQuestions+1):
    FTCorrectFormat[i]={}
    BaselineCorrectFormat[i]={}
    Prompts=[]
    print(i)
    for j in range(0,len(FTOutput)):
        chat=chat2.copy()
        
        if i in FTOutput[j]:
            FTCandidate=str(FTOutput[j][i])
            FTCorrectFormat[i][j]=True
        else:
            FTCandidate=""
            FTCorrectFormat[i][j]=False
            
        if i in BaselineOutput[j]:
            BaselineCandidate=str(BaselineOutput[j][i])
            BaselineCorrectFormat[i][j]=True
        else:
            BaselineCandidate=""
            BaselineCorrectFormat[i][j]=False
        user=""
        if j%2==0:
          user="You are given a comment and a question about the comment. You will be given the response of two systems, system A and system B and you need to judge which system  gave the best response or if it was a tie. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + Comment[j]+".\nHere is the question:\n"+FixedQuestions[i]+"\nHere is the answer of System A:\n" + FTCandidate +"\nHere is the answer of System B:\n"+BaselineCandidate + "\n"
        else:
          user="You are given a comment and a question about the comment. You will be given the response of two systems, system A and system B and you need to judge which system  gave the best response or if it was a tie. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + Comment[j]+".\nHere is the question:\n"+FixedQuestions[i]+"\nHere is the answer of System A:\n" + BaselineCandidate +"\nHere is the answer of System B:\n"+FTCandidate + "\n"

        chat.append({"role": "user", "content": user})
        chat.append({"role": "assistant", "content": Completion})
        
        Prompt=tokenizer.apply_chat_template(chat, tokenize=False)
        Prompt=tokenizer.decode(tokenizer.encode(Prompt)[:-RemoveAssistant])
        Prompts.append(Prompt)
    AllPrompts.append(Prompts)
    


print(tokenizer.encode(Prompts[1]))

        
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
[100264, 9125, 100266, 2675, 527, 264, 11190, 18328, 13, 100265, 100264, 882, 100266, 2675, 527, 2728, 264, 4068, 323, 264, 3488, 922, 279, 4068, 13, 1472, 690, 387, 2728, 279, 2077, 315, 1403, 6067, 323, 499, 1205, 311, 11913, 902, 1887, 220, 6688, 279, 1888, 2077, 13, 7557, 2771, 499, 3493, 279, 4320, 304, 3024, 323, 701, 5873, 374, 2728, 304, 264, 2115, 449, 279, 907, 5873, 13, 5810, 374, 279, 4068, 25, 720, 13347, 9365, 2128, 11, 584, 2351, 25051, 9200, 5178, 4819, 449, 1057, 69278, 20126, 304, 1057, 5788, 4676, 13, 2052, 20126, 527, 4737, 220, 18, 12, 19, 3115, 5129, 1109, 13783, 311, 9203, 11, 35906, 74055, 1057, 2626, 7677, 13, 3053, 499, 4587, 89690, 420, 311, 264, 1579, 10844, 1162, 323, 617, 4423, 1427, 1139, 420, 67590, 30, 1226, 1205, 14247, 13291, 439, 420, 374, 28987, 1057, 17216, 2170, 35047, 8586, 374, 279, 3488, 512, 22186, 420, 4068, 29243, 311, 264, 6130, 12458, 30, 320, 10611, 7887, 8586, 374, 279, 4320, 315, 744, 362, 512, 

In [18]:
#LLM-as-a-judge prompt example
print(AllPrompts[0][3])

<|im_start|>system<|im_sep|>You are a helpful assistant.<|im_end|><|im_start|>user<|im_sep|>You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: 
Hi Support team, we're experiencing critical performance issues with our Hive queries in our production environment. All queries are taking 3-4 times longer than usual to execute, severely impacting our business operations. Can you please escalate this to a high priority case and have someone look into this ASAP? We need immediate assistance as this is affecting our SLAs..
Here is the question:
Does this comment relate to a customer complaint? (BOOL):

Here is the answer of System A:
0
Here is the answer of System B:
1<|im_end|><|im_start|>assistant<|im_sep|>```json
{
    "choice": "B"
}
```<|im_end|><|im_start|>us

In [19]:
#Load LLM-as-a-judge
max_num_seqs=1
max_model_len=10000
max_num_batched_tokens=10000 
gpu_memory_utilization=0.7
enforce_eager=True
tensor_parallel_size=1
seed=None
max_tokens=10
temperature=0.1

import vllm
llm = vllm.LLM(model=EvalLLM, max_num_seqs=max_num_seqs,max_model_len=max_model_len,max_num_batched_tokens=max_num_batched_tokens, gpu_memory_utilization=gpu_memory_utilization,enforce_eager=enforce_eager, tensor_parallel_size=tensor_parallel_size)
seed=None
max_tokens=max_tokens
temperature=temperature
sampling_params = vllm.SamplingParams(seed=None,max_tokens=max_tokens,temperature=temperature)


WARNING 03-21 19:25:00 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-21 19:25:00 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=10000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False)

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  17% Completed | 1/6 [00:00<00:03,  1.40it/s]
Loading safetensors checkpoint shards:  33% Completed | 2/6 [00:01<00:02,  1.34it/s]
Loading safetensors checkpoint shards:  50% Completed | 3/6 [00:02<00:02,  1.34it/s]
Loading safetensors checkpoint shards:  67% Completed | 4/6 [00:02<00:01,  1.36it/s]
Loading safetensors checkpoint shards:  83% Completed | 5/6 [00:03<00:00,  1.28it/s]
Loading safetensors checkpoint shards: 100% Completed | 6/6 [00:04<00:00,  1.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 6/6 [00:04<00:00,  1.32it/s]



INFO 03-21 19:25:07 model_runner.py:1071] Loading model weights took 27.3875 GB
INFO 03-21 19:25:09 gpu_executor.py:122] # GPU blocks: 702, # CPU blocks: 1310
INFO 03-21 19:25:09 gpu_executor.py:126] Maximum concurrency for 10000 tokens per request: 1.12x


In [20]:
#Run LLM-as-a-judge for eacb question and at the same time count correct answers/ties.
Outs=[]
PosPercNoTies=[]
NegPercNoTies=[]
PosPercTies=[]
NegPercTies=[]
countPosArr=[]
countNegArr=[]
countTieArr=[]
TiesPerc=[]

for j in range(0,ExpectedQuestions):
  max_num_seqs=16

  sampling_params = vllm.SamplingParams(seed=1,max_tokens=max_tokens,temperature=temperature)

  Out=llm.generate(AllPrompts[j], sampling_params)
  Outs.append(Out)
  countPos=0
  countNeg=0
  countTie=0

  for i in range(0,len(Out)):
    if i%2==0:
      #print(i)
      #print(Out[i].prompt)
      #print(Out[i].outputs[0].text[2])
      if len(Out[i].outputs[0].text)<2:
        continue

      if Out[i].outputs[0].text[2]=='A':
        countPos+=1
      elif Out[i].outputs[0].text[2]=='B':
        countNeg+=1
      elif Out[i].outputs[0].text[2]=='t':
        countTie+=1

    if i%2==1:
      if len(Out[i].outputs[0].text)<2:
        continue

      if Out[i].outputs[0].text[2]=='B':
        countPos+=1
      elif Out[i].outputs[0].text[2]=='A':
        countNeg+=1
      elif Out[i].outputs[0].text[2]=='t':
        countTie+=1
  print("===================")
  print("Question "+str(j+1))
  print("===================")
  print(str(countPos/(countPos+countNeg)*100)+"%")
  print(str(countNeg/(countPos+countNeg)*100)+"%")
  print(countPos/(countPos+countNeg+countTie))
  print(countNeg/(countPos+countNeg+countTie))

  print(countPos)
  print(countNeg)
  print(countTie)
  PosPercNoTies.append(str(countPos/(countPos+countNeg)*100)+"%")
  NegPercNoTies.append(str(countNeg/(countPos+countNeg)*100)+"%")
  TiesPerc.append(countTie/(countPos+countNeg+countTie))
  PosPercTies.append(countPos/(countPos+countNeg+countTie))
  NegPercTies.append(countNeg/(countPos+countNeg+countTie))
  countPosArr.append(countPos)
  countNegArr.append(countNeg)
  countTieArr.append(countTie)

  

Processed prompts: 100%|█| 500/500 [02:48<00:00,  2.97it/s, est. speed input: 24


Question 1
51.06382978723404%
48.93617021276596%
0.048
0.046
24
23
453


Processed prompts: 100%|█| 500/500 [02:47<00:00,  2.98it/s, est. speed input: 24


Question 2
25.0%
75.0%
0.016
0.048
8
24
468


Processed prompts: 100%|█| 500/500 [02:52<00:00,  2.90it/s, est. speed input: 24


...

Question 15
37.5%
62.5%
0.012
0.02
6
10
484


Processed prompts: 100%|█| 500/500 [02:48<00:00,  2.96it/s, est. speed input: 24


Question 16
33.33333333333333%
66.66666666666666%
0.01
0.02
5
10
485


Processed prompts: 100%|█| 500/500 [02:55<00:00,  2.85it/s, est. speed input: 25

Question 17
89.0909090909091%
10.909090909090908%
0.49
0.06
245
30
225


In [21]:
#Compute winrate
for i in PosPercNoTies:
    print(i)

51.06382978723404%
25.0%
14.975845410628018%
8.304498269896193%
10.104529616724738%
29.155313351498634%
0.0%
40.0%
23.809523809523807%
20.0%
13.333333333333334%
40.0%
50.877192982456144%
0.0%
37.5%
33.33333333333333%
89.0909090909091%


In [22]:
#Compute lose rate!
for i in NegPercNoTies:
    print(i)

48.93617021276596%
75.0%
85.02415458937197%
91.6955017301038%
89.89547038327527%
70.84468664850137%
100.0%
60.0%
76.19047619047619%
80.0%
86.66666666666667%
60.0%
49.122807017543856%
100.0%
62.5%
66.66666666666666%
10.909090909090908%


In [23]:
#Compute ties.
for i in TiesPerc:
    print(str(i*100) + "%")

0.906
0.936
0.586
0.422
0.426
0.266
0.964
0.97
0.958
0.97
0.97
0.97
0.886
0.97
0.968
0.97
0.45
